In [12]:
import pandas as pd

summer = pd.read_csv("../AssB_Input_Group4_summer.csv", index_col = "TimeStamp")
summer.columns

Index(['Start date/time', 'End date/time', 'Residential load [kW]',
       'PV generation [kW]', 'Electricity price [euro/kWh]',
       'Marginal emission factor [kg CO2eq/kWh]'],
      dtype='object')

In [14]:
summer

,Start date/time,End date/time,Residential load [kW],PV generation [kW],Electricity price [euro/kWh],Marginal emission factor [kg CO2eq/kWh]
TimeStamp,,,,,,
0,07/07/2018 00:00,07/07/2018 00:15,0.36722,0.0,0.04771,0.385803
1,07/07/2018 00:15,07/07/2018 00:30,0.34986,0.0,0.04771,0.385803
2,07/07/2018 00:30,07/07/2018 00:45,0.33334,0.0,0.04771,0.385803
3,07/07/2018 00:45,07/07/2018 01:00,0.31794,0.0,0.04771,0.385803
4,07/07/2018 01:00,07/07/2018 01:15,0.30296,0.0,0.04580,0.365847
...,...,...,...,...,...,...
283,09/07/2018 22:45,09/07/2018 23:00,0.42014,0.0,0.05259,0.416062
284,09/07/2018 23:00,09/07/2018 23:15,0.41790,0.0,0.04937,0.955588
285,09/07/2018 23:15,09/07/2018 23:30,0.41048,0.0,0.04937,0.955588
